# Chapter 7 - Classifying Desk Objects with TensorFlow and the Arduino Nano

### Python libraries

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from google.colab import drive

### Constants

In [ ]:
MODEL_ALPHA = 0.35
MODEL_INPUT_WIDTH = 48
MODEL_INPUT_HEIGHT = 48
TFL_MODEL_FILE = "model.tflite"
TFL_MODEL_HEADER_FILE = "model.h"
TF_MODEL = "desk_objects_recognition"

## Transfer learning with Keras

### Mount the top-level Google Drive directory

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


### Prepare the train (80%) and validation (20%) datasets

In [ ]:
# CHANGE TO DESIRED TRAIN FOLDER
train_dir = "drive/MyDrive/dataset"

ds = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  validation_split=0.2,
  subset="both",
  seed=123,
  interpolation="bilinear",
  image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT)
  )

train_ds = ds[0]
val_ds   = ds[1]

Found 60 files belonging to 3 classes.
Using 48 files for training.
Using 12 files for validation.


### Get the number of classes from the training dataset

In [ ]:
class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

['book', 'mug', 'unknown']


### Rescale the pixel values from [0, 255] tp [-1, 1]

In [ ]:
rescale = tf.keras.layers.Rescaling(1./255, offset= -1)
train_ds = train_ds.map(lambda x, y: (rescale(x), y))
val_ds   = val_ds.map(lambda x, y: (rescale(x), y))

### Import the MobileNet v2 pre-trained model

In [ ]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet_v2.py
base_model = MobileNetV2(input_shape=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT, 3),
                         include_top=False,
                         weights='imagenet',
                         alpha=0.35)

2019640/2019640 [==============================] - 0s 0us/step


### Freeze the weights

In [ ]:
base_model.trainable = False
feat_extr = base_model

print("num. weights:", len(base_model.weights))
print("num. trainable_weights:", len(base_model.trainable_weights))
print("num. non_trainable_weights:", len(base_model.non_trainable_weights))

num. weights: 260
num. trainable_weights: 0
num. non_trainable_weights: 260


### Augment the input data

In [ ]:
preproc = tf.keras.layers.experimental.preprocessing
augmen = tf.keras.Sequential([
  preproc.RandomRotation(0.2),
  preproc.RandomFlip('horizontal'),
])

train_ds = train_ds.map(lambda x, y: (augmen(x), y))
val_ds   = val_ds.map(lambda x, y: (augmen(x), y))

### Prepare the classification head

In [ ]:
layers = tf.keras.layers
global_avg_layer = layers.GlobalAveragePooling2D()
dense_layer = layers.Dense(num_classes, activation='softmax')

### Finalize the model architecture

In [ ]:
x = global_avg_layer(feat_extr.layers[-1].output)
x = layers.Dropout(0.2)(x)
outputs = dense_layer(x)
model = tf.keras.Model(inputs=feat_extr.inputs, outputs=outputs)

### Compile the model with a 0.0005 learning rate

In [ ]:
lr = 0.0005
opt_f = tf.keras.optimizers.Adam(learning_rate=lr)
loss_f = tf.losses.SparseCategoricalCrossentropy(from_logits=False)

model.compile(
  optimizer=opt_f,
  loss=loss_f,
  metrics=['accuracy'])

### Model summary

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48, 48, 3)]          0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 24, 24, 16)           432       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 24, 24, 16)           64        ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 24, 24, 16)           0         ['bn_Conv1[0][0]']        

### Train the model with 10 epochs

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

### Save the TensorFlow model

In [ ]:
model.save(TF_MODEL)

## Quantizing and testing the trained model with TensorFlow Lite

### Acquire new images for the test dataset

In [ ]:
# CHANGE TO DESIRED TEST DATASET FOLDER
test_dir = "drive/MyDrive/test"

### Rescale the pixel values from [0, 255] to [-1, 1]

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                      interpolation="bilinear",
                                                      image_size=(MODEL_INPUT_WIDTH, MODEL_INPUT_HEIGHT))
test_ds  = test_ds.map(lambda x, y: (rescale(x), y))

### Quantize the TensorFlow model with the TensorFlow Lite converter

In [ ]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]

converter = tf.lite.TFLiteConverter.from_saved_model(TF_MODEL)
converter.representative_dataset = tf.lite.RepresentativeDataset(representative_data_gen)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8

tfl_model = converter.convert()

### Initialize the TensorFlow Lite interpreter

In [ ]:
# Initialize the TFLite interpreter
interp = tf.lite.Interpreter(model_content=tfl_model)

### Allocate the tensor and get input quantization parameters

In [ ]:
# Allocate the tensors
interp.allocate_tensors()

# Get input/output layer information
i_details = interp.get_input_details()[0]
o_details = interp.get_output_details()[0]

# Get input quantization parameters.
i_quant = i_details["quantization_parameters"]
i_scale      = i_quant['scales'][0]
i_zero_point = i_quant['zero_points'][0]

### Evaluate the accuracy of the quantized TensorFlow Lite model

In [ ]:
test_ds0 = test_ds.unbatch()

num_correct_samples = 0
num_samples   = len(list(test_ds0.batch(1)))

for i_value, o_value in test_ds0.batch(1):
  i_value = (i_value / i_scale) + i_zero_point
  i_value = tf.cast(i_value, dtype=tf.int8)
  interp.set_tensor(i_details["index"], i_value)
  interp.invoke()
  o_pred = interp.get_tensor(o_details["index"])[0]

  if np.argmax(o_pred) == o_value:
    num_correct_samples += 1

print("Accuracy:", num_correct_samples/num_samples)

### Convert the TensorFlow model to C-byte array with xxd

In [ ]:
open(TFL_MODEL_FILE, "wb").write(tfl_model)

In [ ]:
!apt-get update && apt-get -qq install xxd
!xxd -c 60 -i $TFL_MODEL_FILE > $TFL_MODEL_HEADER_FILE
!sed -i 's/unsigned char/const unsigned char/g' $TFL_MODEL_HEADER_FILE
!sed -i 's/const/alignas(8) const/g' $TFL_MODEL_HEADER_FILE

### Get the TensorFlow model size in bytes to estimate the program memory usage

In [ ]:
size_tfl_model = len(tfl_model)
print(len(tfl_model), "bytes")